# Анализ качества данных мошеннических транзакций
**Домашнее задание №3 - Задание №4**  
**Курс MLOps, Otus**

## Описание задания
Ссылка на полный текст: https://raw.githubusercontent.com/OtusTeam/MLOps/refs/heads/main/hw_03/README.md
Основные проблемы, которые могут встречаться в данных, описаны в статье: [Data cleansing - Data quality](https://en.wikipedia.org/wiki/Data_cleansing#Data_quality)

---

## Цель анализа
Проанализировать датасет мошеннических финансовых транзакций на наличие проблем с качеством данных и выявить минимум 3 типа ошибок.

## Структура анализа
1. **Этап 1:** Базовая диагностика данных
2. **Этап 2:** Полнота данных (Completeness)  
3. **Этап 3:** Валидность данных (Validity)
4. **Этап 4:** Дубликаты (Duplicates)
5. **Этап 5:** Точность и выбросы (Accuracy & Outliers)
6. **Этап 6:** Согласованность (Consistency)
7. **Этап 7:** Итоговый отчет

---

## Описание датасета

Датасет содержит информацию о финансовых транзакциях с полями:
- `transaction_id` - уникальный идентификатор транзакции
- `tx_datetime` - дата и время транзакции
- `customer_id` - идентификатор клиента
- `terminal_id` - идентификатор терминала
- `tx_amount` - сумма транзакции
- `tx_time_seconds` - время в секундах
- `tx_time_days` - день транзакции
- `tx_fraud` - флаг мошенничества (0/1)
- `tx_fraud_scenario` - сценарий мошенничества

---
# Этап 1: Базовая диагностика данных

На этом этапе мы:
- Инициализируем Spark
- Загрузим данные из txt файла
- Посмотрим на структуру данных
- Получим базовую статистику
- Поймем объем датасета

## 1.1. Инициализация Spark

In [5]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create Spark session
spark = (
    SparkSession
        .builder
        .appName("OTUS_HW03_DataQuality")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)

print("Spark successfully initialized")
print(f"Spark version: {spark.version}")


Spark successfully initialized
Spark version: 3.0.3


## 1.2. Загрузка и подготовка данных

Загружаем данные из исходного txt файла и правильно определяем типы данных.

**Важно для анализа качества:** На этапе загрузки RAW данных поле `tx_datetime` загружается как `StringType`. Конвертация в `TimestampType` будет выполнена на Этапе 3 (Валидность данных) после анализа формата дат.

### Почему важно правильно определить типы:
- **StringType** для дат на этапе загрузки - сохраняет все значения для анализа
- **LongType** для больших чисел (ID транзакций, секунды)
- **IntegerType** для маленьких чисел (ID клиентов, терминалов, флаги)
- **DoubleType** для денежных сумм
- Правильные типы помогут выявить ошибки в данных


In [2]:
# First, let's look at the structure of the source file

!hdfs dfs -cat data/2022-11-04.txt | head -5


# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario
1832792610,2022-11-04 14:22:18,0,53,63.58,101139738,1170,0,0
1832792611,2022-11-04 02:12:24,0,53,92.95,101095944,1170,0,0
1832792612,2022-11-04 12:49:35,3,205,48.88,101134175,1170,0,0
1832792613,2022-11-04 02:40:01,5,383,24.69,101097601,1170,0,0
cat: Unable to write to output stream.


### Определение схемы данных

Из просмотра видно:
- Первая строка - заголовок с разделителем `|`
- Данные с разделителем `,`
- Поля: `transaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario`

Определим **оптимальные типы** для каждого поля:

In [3]:
# Define schema with correct data types
# 
# NOTE: All fields set to nullable=True to allow detecting data quality issues
# This is correct for RAW data analysis where we want to identify problematic records.
#
# In production pipeline, after data cleaning, critical fields should be nullable=False:
# - transaction_id (primary key)
# - tx_datetime (critical timestamp)
# - customer_id, terminal_id (foreign keys)
# - tx_amount (business-critical value)
# - tx_fraud (required flag)
#
# Only tx_fraud_scenario can remain nullable=True (NULL when tx_fraud=0)

schema = StructType([
    StructField("transaction_id", LongType(), True),        # Long for large numbers (transaction IDs)
    StructField("tx_datetime", StringType(), True),         # String first, then convert to Timestamp
    StructField("customer_id", IntegerType(), True),        # Integer for customer ID
    StructField("terminal_id", IntegerType(), True),        # Integer for terminal ID
    StructField("tx_amount", DoubleType(), True),           # Double for monetary amount (can be fractional)
    StructField("tx_time_seconds", LongType(), True),       # Long for seconds since epoch
    StructField("tx_time_days", IntegerType(), True),       # Integer for day number
    StructField("tx_fraud", IntegerType(), True),           # Integer flag 0/1
    StructField("tx_fraud_scenario", IntegerType(), True)   # Integer for scenario number
])


### Чтение данных из txt файла

Читаем файл с учетом:
- Разделитель `,` (запятая)
- Первая строка начинается с `#` (комментарий) - автоматически пропускается с помощью `comment="#"`
- Используем явную схему для контроля типов данных
- `tx_datetime` загружается как String для последующего анализа валидности формата


In [4]:
# Read txt file with defined schema
df = spark.read.csv(
    "data/2022-11-04.txt",
    sep=",",
    schema=schema,
    comment="#"
)

print(f"Records loaded: {df.count():,}")
df.printSchema()

Records loaded: 46,998,983
root
 |-- transaction_id: long (nullable = true)
 |-- tx_datetime: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- terminal_id: integer (nullable = true)
 |-- tx_amount: double (nullable = true)
 |-- tx_time_seconds: long (nullable = true)
 |-- tx_time_days: integer (nullable = true)
 |-- tx_fraud: integer (nullable = true)
 |-- tx_fraud_scenario: integer (nullable = true)



### Проверка диапазонов значений

Проверим диапазоны значений для числовых полей:


In [ ]:
# Check value ranges
print("Value ranges:")
print("=" * 80)

# Numeric fields
df.select(
    min("transaction_id").alias("min_tx_id"),
    max("transaction_id").alias("max_tx_id"),
    min("customer_id").alias("min_customer"),
    max("customer_id").alias("max_customer"),
    min("terminal_id").alias("min_terminal"),
    max("terminal_id").alias("max_terminal")
).show()

df.select(
    min("tx_amount").alias("min_amount"),
    max("tx_amount").alias("max_amount"),
    avg("tx_amount").alias("avg_amount")
).show()

df.groupBy("tx_fraud").count().orderBy("tx_fraud").show()

## 1.3. Базовая информация о датасете

In [ ]:
# Dataset information
print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
print(f"Records: {df.count():,}")
print(f"Columns: {len(df.columns)}")
print(f"\nSchema:")
for col_name, col_type in df.dtypes:
    print(f"  {col_name:25s} {col_type}")

## 1.4. Просмотр примеров данных

Посмотрим на первые и случайные записи, чтобы понять, как выглядят данные.

In [5]:
# Sample data
df.show(10, truncate=False)


+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|transaction_id|tx_datetime        |customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|1832792610    |2022-11-04 14:22:18|0          |53         |63.58    |101139738      |1170        |0       |0                |
|1832792611    |2022-11-04 02:12:24|0          |53         |92.95    |101095944      |1170        |0       |0                |
|1832792612    |2022-11-04 12:49:35|3          |205        |48.88    |101134175      |1170        |0       |0                |
|1832792613    |2022-11-04 02:40:01|5          |383        |24.69    |101097601      |1170        |0       |0                |
|1832792614    |2022-11-04 08:02:05|6          |858        |95.48    |101116925      |1170        |0       |0  

## 1.5. Базовая статистика

Получим статистическое описание данных - это поможет понять распределение значений и выявить первые признаки проблем.

In [6]:
# Statistical description
df.describe().show()
df.select("tx_amount", "tx_time_seconds", "tx_time_days", "tx_fraud").summary().show()


+-------+--------------------+-------------------+-----------------+------------------+-----------------+--------------------+------------------+-------------------+--------------------+
|summary|      transaction_id|        tx_datetime|      customer_id|       terminal_id|        tx_amount|     tx_time_seconds|      tx_time_days|           tx_fraud|   tx_fraud_scenario|
+-------+--------------------+-------------------+-----------------+------------------+-----------------+--------------------+------------------+-------------------+--------------------+
|  count|            46998983|           46998983|         46998983|          46996685|         46998983|            46998983|          46998983|           46998983|            46998983|
|   mean| 1.856292096624556E9|               null| 500437.420257349|28085.477290004605|54.17185979556221|1.0238400246815348E8|1184.5000234154004|0.02991875377388485|0.060363710423265965|
| stddev|1.3567435381965572E7|               null|288558.35183784

## 1.6. Итоги Этапа 1

**Выполнено:**
- Инициализирован Spark
- Загружены данные из txt-файла
- Определены типы данных (tx_datetime — String для анализа)
- Получена базовая статистика

**Критические проблемы качества данных:**
1. Невалидная дата: "2022-12-03 24:00:00"
   - Проблема: час 24:00:00 не существует (должно быть 00:00:00 следующего дня).
   - Количество: ≥ 1 запись (по max).
   - Влияние: при конвертации в TimestampType станет NULL.
   - Действие: найти записи и скорректировать дату на следующий день.

2. Отрицательные customer_id: min = -999999
   - Проблема: идентификатор клиента не может быть отрицательным.
   - Возможная причина: placeholder отсутствующих данных или ошибка импорта.
   - Действие: оценить количество, решить — удалить или заменить на NULL.

3. Нулевые суммы транзакций: min = 0.0
   - Проблема: транзакция без суммы бессмысленна для финансового анализа.
   - Действие: посчитать количество, определить политику (исключить/пометить как тестовые).

4. Сильный выброс в terminal_id: max = 89,518,096
   - Проблема: при среднем ~28,085 максимум в 3,000+ раз больше.
   - Возможная причина: ошибка ввода или некорректный ID.
   - Действие: изучить распределение, выделить и проверить выбросы.

5. Отсутствие данных (расхождение count)
   - tx_datetime: 46,998,983
   - terminal_id: 46,996,685 (−2,298)
   - Проблема: NULL в критичном поле terminal_id.
   - Действие: посчитать NULL по всем полям и выявить причины.

6. Доля мошенничества: 2.99%
   - Норма: обычно 0.1–1% в реальных данных.
   - Вывод: датасет синтетический или предфильтрованный.
   - Действие: учесть дисбаланс классов при ML.

**Что проверить далее:**
- NULL в terminal_id — объем и причины
- Отрицательные customer_id — паттерн или случайность
- Нулевые суммы — валидные операции или ошибки
- Выбросы terminal_id — топ‑10 наибольших
- Даты вида "24:00:00" — количество записей
- Дубликаты transaction_id

**Следующий этап:** Анализ полноты данных (Completeness)


---
# Этап 3: Валидность данных (Validity)

На этом этапе проверяем корректность формата и значений данных. Особое внимание уделяем полю `tx_datetime`, которое было загружено как String.

## Задачи:
1. Проверить формат дат в поле tx_datetime
2. Выявить невалидные значения
3. Подсчитать количество проблем
4. Конвертировать валидные даты в TimestampType


## 3.1. Анализ формата дат в tx_datetime

Проверим различные проблемы с датами:
- NULL значения
- Пустые строки
- Невалидный формат (не соответствует "yyyy-MM-dd HH:mm:ss")


In [ ]:
# Create parsed timestamp column (NULL if format invalid)
df_with_parsed = df.withColumn(
    "tx_datetime_parsed",
    to_timestamp("tx_datetime", "yyyy-MM-dd HH:mm:ss")
)

total_records = df.count()

# Count NULL in original data
null_original = df.filter(col("tx_datetime").isNull()).count()

# Count empty strings
empty_strings = df.filter(col("tx_datetime") == "").count()

# Count invalid format (originally not NULL, but failed to parse)
invalid_format = df_with_parsed.filter(
    col("tx_datetime").isNotNull() &
    (col("tx_datetime") != "") &
    col("tx_datetime_parsed").isNull()
).count()

# Count successfully parsed
valid_dates = df_with_parsed.filter(col("tx_datetime_parsed").isNotNull()).count()

print("=" * 80)
print("DATE VALIDITY ANALYSIS")
print("=" * 80)
print(f"Total records:        {total_records:,}")
print(f"Valid dates:          {valid_dates:,} ({valid_dates/total_records*100:.2f}%)")
print(f"NULL values:          {null_original:,}")
print(f"Empty strings:        {empty_strings:,}")
print(f"Invalid format:       {invalid_format:,}")
print(f"Total problems:       {null_original + empty_strings + invalid_format:,}")

pass


## 3.2. Примеры невалидных дат

Если найдены проблемы, посмотрим на примеры:


In [ ]:
# Show examples of problematic dates
if invalid_format > 0:
    print("Examples of invalid date formats:")
    df_with_parsed.filter(
        col("tx_datetime").isNotNull() &
        (col("tx_datetime") != "") &
        col("tx_datetime_parsed").isNull()
    ).select("transaction_id", "tx_datetime").show(10, truncate=False)

if empty_strings > 0:
    print("Examples of empty strings:")
    df.filter(col("tx_datetime") == "").select("transaction_id", "tx_datetime").show(5)

if null_original > 0:
    print(f"NULL values found: {null_original}")

pass


## 3.3. Конвертация в TimestampType

После анализа конвертируем tx_datetime в правильный тип. Записи с невалидными датами будут иметь NULL в поле tx_datetime.


In [ ]:
# Apply timestamp conversion
df = df.withColumn(
    "tx_datetime",
    to_timestamp("tx_datetime", "yyyy-MM-dd HH:mm:ss")
)

print("tx_datetime converted to TimestampType")
df.printSchema()

# Verify conversion
df.select(
    min("tx_datetime").alias("earliest"),
    max("tx_datetime").alias("latest")
).show(truncate=False)

pass


## 3.4. Другие проверки валидности

Проверим валидность других полей:


In [ ]:
# Apply production schema to cleaned data
try:
    df_production = spark.createDataFrame(df_clean.rdd, schema_production)
    
    print("Production DataFrame created successfully")
    print("=" * 60)
    print(f"Production records: {df_production.count():,}")
    
    df_production.printSchema()
    df_production.show(5, truncate=False)
    
except Exception as e:
    print("ERROR: Failed to create production DataFrame")
    print(f"Error: {str(e)}")
    print("Cleaned data contains NULLs in nullable=False fields")

pass


## 3.5. Итоги Этапа 3

**Проверено:**
- Формат дат в tx_datetime
- Отрицательные суммы транзакций
- Невалидные значения флагов
- Отрицательные значения дней

**Результаты анализа валидности сохранены для итогового отчёта.**

**Следующий этап:** Анализ дубликатов (Duplicates)


# Save production data to HDFS in Parquet format
# Uncomment and adjust path as needed

# output_path = "hdfs:///user/data/transactions_clean"

# df_production.write \
#     .mode("overwrite") \
#     .partitionBy("tx_time_days") \
#     .parquet(output_path)

# print(f"Production data saved to {output_path}")
# print(f"Format: Parquet")
# print(f"Records: {df_production.count():,}")

print("Section 8.4: Code example (commented)")

pass


## 8.5. Итоги Этапа 8

**Выполнено:**
1. Очищены данные от проблемных записей (NULL, дубликаты, выбросы)
2. Создана строгая production-схема с `nullable=False` для критичных полей
3. Применена схема к очищенным данным (с защитой от битых данных)
4. Подготовлен код для сохранения в production хранилище

**Ключевые отличия RAW vs Production:**

| Аспект | RAW (Этап 1) | Production (Этап 8) |
|--------|--------------|---------------------|
| Цель | Анализ качества | Использование в системах |
| `nullable` | `True` (все поля) | `False` (критичные поля) |
| Данные | Все, включая битые | Только валидные |
| Защита | Нет | Схема защищает от NULL |

**Когда применять каждую схему:**

**RAW схема (`nullable=True`):**
- Загрузка сырых данных
- Анализ качества
- Debugging и исследование проблем

**Production схема (`nullable=False`):**
- После очистки и валидации
- Перед сохранением в БД/хранилище
- Для использования в ML/аналитике
- В API и сервисах

**Важно для производственных систем:**
- Строгая схема - первая линия защиты data integrity
- Не полагайтесь только на логику очистки - схема должна гарантировать качество
- В критичных системах используйте дополнительные проверки (constraints, triggers)

---

**Навигация:** Этот этап является бонусным и показывает полный lifecycle обработки данных. Для выполнения домашнего задания достаточно Этапов 1-7 (анализ качества). Этап 8 демонстрирует понимание production best practices.


In [ ]:
# Clean data based on analysis results from Stages 2-7
print("Data Cleaning")
print("=" * 60)

print(f"Original: {df.count():,}")

# Remove NULLs in critical fields
df_clean = df.filter(
    col("transaction_id").isNotNull() &
    col("tx_datetime").isNotNull() &
    col("customer_id").isNotNull() &
    col("terminal_id").isNotNull() &
    col("tx_amount").isNotNull() &
    col("tx_fraud").isNotNull()
)
print(f"After NULL removal: {df_clean.count():,}")

# Remove duplicates
df_clean = df_clean.dropDuplicates(["transaction_id"])
print(f"After deduplication: {df_clean.count():,}")

# Handle tx_fraud_scenario NULLs
df_clean = df_clean.fillna({"tx_fraud_scenario": 0})

# Remove invalid values
df_clean = df_clean.filter(
    (col("tx_amount") > 0) &
    (col("tx_fraud").isin([0, 1])) &
    (col("tx_time_days") >= 0)
)
print(f"After validation: {df_clean.count():,}")

removed = df.count() - df_clean.count()
print(f"Total removed: {removed:,} ({removed/df.count()*100:.2f}%)")

pass


## 8.2. Создание Production-Ready схемы

После очистки данных создаём **строгую схему** с `nullable=False` для критичных полей.

### Ключевые отличия от RAW-схемы:

| Поле | RAW (Этап 1) | Production (Этап 8) | Обоснование |
|------|--------------|---------------------|-------------|
| `transaction_id` | `nullable=True` | **`nullable=False`** | Первичный ключ |
| `tx_datetime` | `nullable=True` | **`nullable=False`** | Критичная временная метка |
| `customer_id` | `nullable=True` | **`nullable=False`** | Обязательная привязка к клиенту |
| `terminal_id` | `nullable=True` | **`nullable=False`** | Обязательная привязка к терминалу |
| `tx_amount` | `nullable=True` | **`nullable=False`** | Бизнес-критичное значение |
| `tx_fraud` | `nullable=True` | **`nullable=False`** | Обязательный флаг |
| `tx_fraud_scenario` | `nullable=True` | **`nullable=True`** | NULL допустим (нет мошенничества) |


In [ ]:
# Define production schema with strict nullable constraints
schema_production = StructType([
    StructField("transaction_id", LongType(), False),
    StructField("tx_datetime", TimestampType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("terminal_id", IntegerType(), False),
    StructField("tx_amount", DoubleType(), False),
    StructField("tx_time_seconds", LongType(), False),
    StructField("tx_time_days", IntegerType(), False),
    StructField("tx_fraud", IntegerType(), False),
    StructField("tx_fraud_scenario", IntegerType(), True)
])

print("Production schema:")
print("=" * 60)
for field in schema_production.fields:
    nullable = "NULL" if field.nullable else "NOT NULL"
    print(f"{field.name:25s} {str(field.dataType):20s} {nullable}")

pass


## 8.3. Применение строгой схемы к очищенным данным

Теперь применяем production-схему к очищенным данным. 

**Важно:** Если после очистки остались NULL в полях с `nullable=False`, Spark выбросит ошибку - это **защита** от сохранения битых данных в production!


In [ ]:
# Apply strict production schema to cleaned data
try:
    # Create production DataFrame with strict schema
    df_production = spark.createDataFrame(df_clean.rdd, schema_production)
    
    print("✅ Production DataFrame created successfully!")
    print("=" * 60)
    print(f"Production-ready records: {df_production.count():,}")
    
    # Verify schema
    print("\nProduction DataFrame Schema:")
    df_production.printSchema()
    
    # Show sample data
    print("\nSample production data:")
    df_production.show(5, truncate=False)
    
except Exception as e:
    print("❌ ERROR: Failed to create production DataFrame!")
    print(f"Error: {str(e)}")
    print("\nThis means cleaned data still contains NULLs in fields marked as nullable=False")
    print("Review and improve data cleaning logic in section 8.1")

pass


## 8.4. Сохранение в Production хранилище

После создания production-ready DataFrame, данные можно сохранить в production хранилище (HDFS, S3, Delta Lake и т.д.)

**Рекомендуемые форматы:**
- **Parquet** - эффективное сжатие, поддержка схемы, быстрое чтение
- **Delta Lake** - ACID transactions, versioning, time travel
- **ORC** - оптимизирован для Hive, хорошее сжатие


In [ ]:
# Example: Save production-ready data to HDFS in Parquet format
# Uncomment and adjust path as needed

# output_path = "hdfs:///user/data/transactions_clean"

# df_production.write \
#     .mode("overwrite") \
#     .partitionBy("tx_time_days") \
#     .parquet(output_path)

# print(f"✅ Production data saved to {output_path}")
# print(f"Format: Parquet (compressed)")
# print(f"Partitioned by: tx_time_days")
# print(f"Records saved: {df_production.count():,}")

print("💡 Section 8.4: Code example provided (commented out)")
print("Uncomment and adjust the path when ready to save production data")


## 8.5. Итоги Этапа 8

**Что мы сделали:**
1. ✅ Очистили данные от проблемных записей (NULL, дубликаты, выбросы)
2. ✅ Создали **строгую production-схему** с `nullable=False` для критичных полей
3. ✅ Применили схему к очищенным данным (с защитой от битых данных)
4. ✅ Подготовили код для сохранения в production хранилище

**Ключевые отличия RAW vs Production:**

| Аспект | RAW (Этап 1) | Production (Этап 8) |
|--------|--------------|---------------------|
| Цель | Анализ качества | Использование в системах |
| `nullable` | `True` (все поля) | `False` (критичные поля) |
| Данные | Все, включая битые | Только валидные |
| Защита | Нет | Схема защищает от NULL |

**Когда применять каждую схему:**

📊 **RAW схема (`nullable=True`):**
- Загрузка сырых данных
- Анализ качества
- Debugging и исследование проблем

🔒 **Production схема (`nullable=False`):**
- После очистки и валидации
- Перед сохранением в БД/хранилище
- Для использования в ML/аналитике
- В API и сервисах

**Важно для производственных систем:**
- Строгая схема - это первая линия защиты data integrity
- Не полагайтесь только на логику очистки - схема должна гарантировать качество
- В критичных системах используйте дополнительные проверки (constraints, triggers и т.д.)

---

**Навигация:** Этот этап является **бонусным** и показывает полный lifecycle обработки данных. Для выполнения домашнего задания достаточно Этапов 1-7 (анализ качества). Этап 8 демонстрирует понимание production best practices.
